In [16]:
import numpy as np
from hybrid_mdmc.interactions import *
from hybrid_mdmc.system import *

Create a hybrid_mdmc.system.SystemData instance. The NAME and PREFIX can be changed to whatever you like.

In [17]:
my_system_data = SystemData(name="kmctoy", prefix="kmctoy")

Fill in the Hybrid kMC/MD settings. These settings affect various hyperparameters as well as reaction parameters.

In [18]:
my_system_data.hkmcmd = {
    "temperature_rxn": 126.48, # reaction temperature
    "temperature_rxn_units": "K", # units of the above temperature
    "diffusion_cutoff": 0.0, # minimum diffusion rate between the voxels containing the two reactants of a bimolecular reaction for their combination to be considered a reaction candidate
    "change_threshold": 0.05, # the reactive kMC loop ends after (number of initial molecules * change_threshold) molecules have reacted
    "scale_rates": False, # whether or not to scale the rates of the reactions
    "avoid_double_counts": True, # whether or not to avoid counting i + j -> ij and i + j -> ji as two separate reactions
    "msd_start": 0, # starting frame for MSD calculation
    "msd_every": 1, # calculate MSD every msd_every frames
    "msd_end": -1, # ending frame for MSD calculation
}

Fill in the reaction scaling settings. These settings affect how psuedo-steady-state is identified and how reaction rates are subsequently scaled. If reaction rate scaling is not being applied, scripts within hybrid_mdmc will ignore these settings.

In [19]:
my_system_data.scaling_reaction = {
    "concentration_cycles": 1e99, # number of cycles over which the slope of the concentration must be less than or equal to the threshold
    "concentration_slope": 0.0, # threshold slope of the concentration
    "reaction_selection_count": 1e99, # number of times the reaction must have been selected in the defined window
    "windowsize_rxnselection": 1e99, # window over which to count the number of reaction selections
    "windowsize_slope": 1e99, # window over which to calculate the slope of the concentration
    "windowsize_scalingpause": 1e99, # number of frames over which to pause scaling
    "scaling_factor": 1.0, # factor by which to scale the reaction rate
    "scaling_minimum": 1.0  # minimum value of the scaling factor
}

Fill in the diffuion scaling settings. These settings dictate how diffusion rates between voxels are calculated.  Interpretation of many of these settings requires reading through calculate_diffusion.py. If diffusion scaling is not being applied, "well-mixed" should be set to True, and scripts within hybrid_mdmc will ignore the other settings.

In [27]:
my_system_data.scaling_diffusion = {
    "number_of_voxels": [6,6,6], # number of voxels in each dimension
    "x_bounds": None, # voxel bounds in x dimension (overrides number_of_voxels if provided)
    "y_bounds": None, # voxel bounds in y dimension (overrides number_of_voxels if provided)
    "z_bounds": None, # voxel bounds in z dimension (overrides number_of_voxels if provided)
    "direct_transition_rates_method": "fuzzy_boundary", # method for calculating local diffusion rates between voxels, from an MD trajectory; see calculate_difusion.py
    "global_diffusion_rates_method": "random_walk", # method of calculating the global diffusion rates between voxels, from local diffusion rates; see calculate_diffusion.py
    "fuzz": 0.1, # fuzz factor for fuzzy boundary method
    "trj_parse_start": 1, # starting frame for parsing the trajectory
    "trj_parse_every": 1, # parse every trj_parse_every frames
    "trj_parse_end": -1, # ending frame for parsing the trajectory
    "average_across_voxel_neighbors": True, # if True, local diffusion rates will be averaged over all calculated rates between voxels separated by the same distance
    "recursion_interval": 1, # interval over which to iterate while calculating global diffusion rates from the random walk outputs
    "well-mixed": True # if True, the system is assumed to be well-mixed and diffusion scaling is NOT applied
}

Fill in the LAMMPS information. Some of these settings are used when creating the LAMMPS in.data and in.init files, while others are used by hybrid_mdmc scripts in calculations. Definitions of many of these settings are explained in the LAMMPS documentation.

In [30]:
my_system_data.lammps = {
    "units": "lj",
    "LJ_mass": 2.65692461e-26, # reference mass for LJ units
    "LJ_mass_units": "kg",
    "LJ_distance": 3.4e-10, # reference distance for LJ units
    "LJ_distance_units": "m",
    "LJ_energy": 2.04964162e-21, # reference energy for LJ units
    "LJ_energy_units": "J",
    "LJ_stepsize": 0.001, # timestep of LAMMPS diffusion simulation, used for some calculations
    "time_conversion": 1.22e-12, # conversion factor from LAMMPS time units to seconds
    "atom_style": "full",
    "pair_style": "lj/cut 10.0",
    "bond_style": "harmonic",
    "angle_style": "harmonic",
    "dihedral_style": "harmonic",
    "improper_style": "harmonic",
    "atom_types": 2,
    "bond_types": 1,
    "angle_types": 0,
    "dihedral_types": 0,
    "improper_types": 0,
    "starting_A": 400, # beginning number of A atoms
    "starting_A2": 0, # beginning number of A2 atoms
    "starting_B": 1000, # beginning number of B atoms
    "charged_atoms": True # whether or not LAMMPS atom definitions will include charge
}

Define the chemistry of the system. This is kept in the SYSTEM.json file as many scripts require knowledge of system chemistry for various calculations. The system chemistry is held in template hybrid_mdmc.interactions.Molecule objects, one for each molecule type in the system. A Molecule object includes instances of hybrid_mdmc.interactions.Atom and hybrid_mdmc.interactions.IntraMode. A comprehensive list of the system atoms and the system bonded interactions is created from the list of Molecule objects when needed in individual scripts. Molecule definitions in the SystemData.species attribute can be self-containing; i.e. the molecule ID can be "1" and each atom ID can be 1 indexed, they need not be coherent with the system's other Molecule definitions. All the scripts have been written to understand that these are "template" Molecule objects, and will handle everything appropriately. (when creating a list of Molecule objects representing actual molecules of a system, not just the templates, this is NOT the case; molecule IDs, tom IDs, and IntraMode IDs must be coordinated)

In [34]:
A_molecule = Molecule(
    ID=1,
    kind="A",
    atoms=[
        Atom(ID=1, kind=1, molecule_ID=1, molecule_kind="A", lammps_type=1, mass=1.0, x=0.0, y=0.0, z=0.0, charge=0.0),
    ],
    bonds=None,
    angles=None,
    dihedrals=None,
    impropers=None,
)
A2_molecule = Molecule(
    ID=1,
    kind="A2",
    atoms=[
        Atom(ID=1, kind=1, molecule_ID=1, molecule_kind="A2", lammps_type=1, mass=1.0, x=-0.5, y=0.0, z=0.0, charge=0.0),
        Atom(ID=2, kind=1, molecule_ID=1, molecule_kind="A2", lammps_type=1, mass=1.0, x= 0.5, y=0.0, z=0.0, charge=0.0),
    ],
    bonds=[
        IntraMode(ID=1, kind=1, atom_IDs=[1,2]),
    ],
    angles=None,
    dihedrals=None,
    impropers=None,
)
B_molecule = Molecule(
    ID=1,
    kind="B",
    atoms=[
        Atom(ID=1, kind=2, molecule_ID=1, molecule_kind="B", lammps_type=2, mass=2.0, x=0.0, y=0.0, z=0.0, charge=0.0),
    ],
    bonds=None,
    angles=None,
    dihedrals=None,
    impropers=None,
)

my_system_data.species = [A_molecule, A2_molecule, B_molecule]

Define the possible reactions of the system. Each reaction is held as a template hybrid_mdmc.interactions.Reaction object. The objects have attributes that are typical in the definition of a reaction (e.g. activation energy). It also holds a list of Molecule objects as reactants and another for products. In defining a reaction, specific atomic bonds are broken/formed. To respect this, the Molecule objects in the reactants and products list must be written as a cohesive system. This means 
1. atom IDs **cannot** overlap among reactants, nor products
2. the set of atom IDs among the reactants should match exactly to the set of atom IDs among the products
    - no atoms are created or destroyed, only their bonding behavior and their molecular assignment
3. IntraMode IDs **cannot** overlap among reactants, nor products.
4. the set of reactant IntraMode IDs need not be equal tot he set of product IntraMode IDs
    - as bonds are broken and formed, bonds/angles/dihedrals/impropers may be destroyed or created

In [35]:
# Forward reaction, A + A -> A2
reactant_f1 = Molecule(
    ID=1,
    kind="A",
    atoms=[
        Atom(ID=1, kind=1, molecule_ID=1, molecule_kind="A", lammps_type=1, mass=1.0, x=0.0, y=0.0, z=0.0, charge=0.0),
    ],
    bonds=None,
    angles=None,
    dihedrals=None,
    impropers=None,
)
reactant_f2 = Molecule(
    ID=2,
    kind="A",
    atoms=[
        Atom(ID=2, kind=1, molecule_ID=2, molecule_kind="A", lammps_type=1, mass=1.0, x=1.0, y=0.0, z=0.0, charge=0.0),
    ],
    bonds=None,
    angles=None,
    dihedrals=None,
    impropers=None,
)
product_f1 = Molecule(
    ID=3,
    kind="A2",
    atoms=[
        Atom(ID=1, kind=1, molecule_ID=3, molecule_kind="A2", lammps_type=1, mass=1.0, x=0.0, y=0.0, z=0.0, charge=0.0),
        Atom(ID=2, kind=1, molecule_ID=3, molecule_kind="A2", lammps_type=1, mass=1.0, x=1.0, y=0.0, z=0.0, charge=0.0),
    ],
    bonds=[
        IntraMode(ID=1, kind=1, atom_IDs=[1,2]),
    ],
    angles=None,
    dihedrals=None,
    impropers=None,
)
forward_reaction = Reaction(
    ID=1,
    kind=1,
    reactant_molecules=[reactant_f1, reactant_f2],
    product_molecules=[product_f1],
    Ea=4.0511,
    Ea_units="kcal/mol",
    A=1e8,
    A_units="1/s",
    b=0.0,
)

# Reverse reaction, A2 -> A + A
reactant_r1 = Molecule(
    ID=1,
    kind="A2",
    atoms=[
        Atom(ID=1, kind=1, molecule_ID=1, molecule_kind="A2", lammps_type=1, mass=1.0, x=0.0, y=0.0, z=0.0, charge=0.0),
        Atom(ID=2, kind=1, molecule_ID=1, molecule_kind="A2", lammps_type=1, mass=1.0, x=1.0, y=0.0, z=0.0, charge=0.0),
    ],
    bonds=[
        IntraMode(ID=1, kind=1, atom_IDs=[1,2]),
    ],
    angles=None,
    dihedrals=None,
    impropers=None,
)
product_r1 = Molecule(
    ID=2,
    kind="A",
    atoms=[
        Atom(ID=1, kind=1, molecule_ID=2, molecule_kind="A", lammps_type=1, mass=1.0, x=0.0, y=0.0, z=0.0, charge=0.0),
    ],
    bonds=None,
    angles=None,
    dihedrals=None,
    impropers=None,
)
product_r2 = Molecule(
    ID=3,
    kind="A",
    atoms=[
        Atom(ID=2, kind=1, molecule_ID=3, molecule_kind="A", lammps_type=1, mass=1.0, x=1.0, y=0.0, z=0.0, charge=0.0),
    ],
    bonds=None,
    angles=None,
    dihedrals=None,
    impropers=None,
)
reverse_reaction = Reaction(
    ID=2,
    kind=2,
    reactant_molecules=[reactant_r1],
    product_molecules=[product_r1, product_r2],
    Ea=2.8937,
    Ea_units="kcal/mol",
    A=1e8,
    A_units="1/s",
    b=0.0,
)

my_system_data.reactions = [forward_reaction, reverse_reaction]

Fill in the information required to write the LAMMPS in.init file for the initial MD run. This information is used by hybrid_mdmc.filehandlers.LammpsInitHandler to write the in.init file.

In [24]:
my_system_data.MD_initial = {
    "run_name": [
        "shrink",
        "diffusion",
    ],
    "run_style": [
        "nvt deform",
        "nvt",
    ],
    "run_stepsize": [
        0.001,
        0.001,
    ],
    "run_steps": [
        15000,
        50000,
    ],
    "run_temperature": [
        "0.852 0.852 0.425",
        "0.852 0.852 0.425",
    ],
    "run_pressure_volume": [
        "1 x final -5.25 5.25 y final -5.25 5.25 z final -5.25 5.25 units box",
        None,
    ],
    "thermo_freq": 100,
    "coords_freq": 200,
    "avg_calculate_every": 50,
    "avg_number_of_steps": 10,
    "avg_stepsize": 5,
    "units": "lj",
    "atom_style": "full",
    "dimension": 3,
    "newton": "on",
    "pair_style": "lj/cut 3.0",
    "bond_style": "harmonic",
    "angle_style": "harmonic",
    "dihedral_style": "opls",
    "improper_style": "cvff",
    "thermo_keywords": "temp press ke pe ebond evdwl",
    "neigh_modify": "every 5 delay 0 check no one 10000",
    "write_trajectories": True,
    "write_intermediate_restarts": False,
    "write_final_data": True,
    "write_final_restarts": True
}

Fill in the information required to write the LAMMPS in.init file for each diffusive cycle's MD run. This information is used by hybrid_mdmc.filehandlers.LammpsInitHandler to write the in.init file.

In [25]:
my_system_data.MD_cycling = {
    "run_name": [
        "relax",
        "equil",
        "diffusion",
    ],
    "run_style": [
        "nve/limit",
        "nve",
        "nve",
    ],
    "run_stepsize": [
        0.00025,
        0.001,
        0.001,
    ],
    "run_steps": [
        1000,
        2000,
        10000,
    ],
    "run_temperature": [
        "0.852 0.852 0.425",
        "0.852 0.852 0.425",
        "0.852 0.852 0.425",
    ],
    "run_pressure_volume": [
        0.03,
        None,
        None,
    ],
    "thermo_freq": 100,
    "coords_freq": 100,
    "avg_calculate_every": 50,
    "avg_number_of_steps": 10,
    "avg_stepsize": 5,
    "units": "lj",
    "atom_style": "full",
    "dimension": 3,
    "newton": "on",
    "pair_style": "lj/cut 3.0",
    "bond_style": "harmonic",
    "angle_style": "harmonic",
    "dihedral_style": "opls",
    "improper_style": "cvff",
    "thermo_keywords": "temp press ke pe ebond evdwl",
    "neigh_modify": "every 5 delay 0 check no one 10000",
    "write_trajectories": True,
    "write_intermediate_restarts": False,
    "write_final_data": True,
    "write_final_restarts": True
}

Finally, write the SystemDat object to a json file.

In [36]:
my_system_data.write_json()